In [1]:
# RS BPP
!pip install reedsolo
from reedsolo import RSCodec

In [2]:
# Import necessary libraries
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from PIL import Image
import sys

from reedsolo import RSCodec

In [3]:
''' MY FILES '''
from google.colab import drive
drive.mount('/content/drive')

sys.path.append('/content/drive/My Drive/DU/COMP 3432 Machine Learning/Steganography with GANs/Train')
print("Path added to sys.path:", sys.path[-1])
print("Files in the directory:", os.listdir('/content/drive/My Drive/DU/COMP 3432 Machine Learning/Steganography with GANs/Train'))


from loader  import MyDataset
from encoder import BasicMishEncoder, DenseMishEncoder, ResidualMishEncoder
from decoder import BasicMishDecoder, DenseMishDecoder
from critics import BasicMishCritic, DenseMishCritic, ResidualMishCritic
from models  import MISHSteganoGAN

from utils   import bits_to_bytearray, bytearray_to_text, ssim, text_to_bits # SteganoGAN utils file



# Define paths (adjust based on your local setup)
cover_dir = '/content/drive/My Drive/DU/COMP 3432 Machine Learning/Steganography with GANs/Train/Data/cover'
validate_dir =  '/content/drive/My Drive/DU/COMP 3432 Machine Learning/Steganography with GANs/Train/Data/validate'
log_dir = '/content/drive/My Drive/DU/COMP 3432 Machine Learning/Steganography with GANs/Train/logs'

# List all image files in the cover directory
cover_imgs = sorted([os.path.join(cover_dir, f) for f in os.listdir(cover_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
validate_imgs = sorted([os.path.join(validate_dir, f) for f in os.listdir(validate_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
print(len(cover_imgs))
print(len(validate_imgs))

# DATA LOADER
def create_data_loader(cover_imgs, stego_imgs, transform, shuffle=False, num_workers=4, batch_size=4):
  dataset = MyDataset(cover_imgs, stego_imgs, transform)
  return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

Mounted at /content/drive
Path added to sys.path: /content/drive/My Drive/DU/COMP 3432 Machine Learning/Steganography with GANs/Train
Files in the directory: ['utils.py', 'decoder.py', 'encoder.py', 'critics.py', 'loader.py', 'files', 'logs', 'stego', '__pycache__', 'decode', 'Data', 'models.py', 'eval.ipynb', 'train.ipynb']
800
200


In [4]:
# import gc
# torch.cuda.empty_cache()
# gc.collect()


In [5]:
# BASIC ENCODER, DECODER, CRITIC

# Initialize data loader
batch_size = 16 # 32-image batches
train_loader = create_data_loader(cover_imgs, cover_imgs, transform=None, shuffle=True, batch_size=batch_size)
validate_loader = create_data_loader(validate_imgs, validate_imgs, transform=None, shuffle=False, batch_size=batch_size)


# Initialize agents and values
data_depth = 3 # typical for RBG values
hidden_size = 64 #


encoder = BasicMishEncoder(data_depth, hidden_size)
decoder = BasicMishDecoder(data_depth, hidden_size)
critic = BasicMishCritic(hidden_size)


# Initialize the GAN model
model = MISHSteganoGAN(
    data_depth=data_depth,
    encoder=encoder,
    decoder=decoder,
    critic=critic,
    cuda=True,
    log_dir=log_dir,
    verbose=True
)
# print(model)

model.fit(train_loader, validate_loader, epochs=16)

Using CUDA device.
Model initialized with data depth 3
Epoch 1/16


Metrics after epoch 1:
val.encoder_mse: 0.0247
val.decoder_loss: 0.6522
val.decoder_acc: 0.6437
val.cover_score: 0.0245
val.generated_score: 0.0251
val.ssim: 0.5983
val.psnr: 22.3181
val.bpp: 0.8622
train.encoder_mse: 0.0810
train.decoder_loss: 0.6801
train.decoder_acc: 0.5914
train.cover_score: 0.0249
train.generated_score: 0.0272
Epoch 2/16


Metrics after epoch 2:
val.encoder_mse: 0.0186
val.decoder_loss: 0.6261
val.decoder_acc: 0.6725
val.cover_score: 0.0251
val.generated_score: 0.0293
val.ssim: 0.6783
val.psnr: 23.6603
val.bpp: 1.0349
train.encoder_mse: 0.0279
train.decoder_loss: 0.6373
train.decoder_acc: 0.6621
train.cover_score: 0.0249
train.generated_score: 0.0273
Epoch 3/16


Metrics after epoch 3:
val.encoder_mse: 0.0155
val.decoder_loss: 0.5984
val.decoder_acc: 0.6966
val.cover_score: 0.0251
val.generated_score: 0.0354
val.ssim: 0.7114
val.psnr: 24.4480
val.bpp: 1.1796
train.encoder_mse: 0.0247
train.decoder_loss: 0.6106
train.decoder_acc: 0.6866
train.cover_score: 0.0252
train.generated_score: 0.0330
Epoch 4/16


Metrics after epoch 4:
val.encoder_mse: 0.0145
val.decoder_loss: 0.5638
val.decoder_acc: 0.7223
val.cover_score: 0.0254
val.generated_score: 0.0408
val.ssim: 0.7209
val.psnr: 24.7352
val.bpp: 1.3337
train.encoder_mse: 0.0202
train.decoder_loss: 0.5785
train.decoder_acc: 0.7112
train.cover_score: 0.0251
train.generated_score: 0.0385
Epoch 5/16


...

Metrics after epoch 12:
val.encoder_mse: 0.0091
val.decoder_loss: 0.3441
val.decoder_acc: 0.8503
val.cover_score: 0.0240
val.generated_score: 0.0473
val.ssim: 0.7369
val.psnr: 26.9603
val.bpp: 2.1021
train.encoder_mse: 0.0144
train.decoder_loss: 0.3502
train.decoder_acc: 0.8491
train.cover_score: 0.0239
train.generated_score: 0.0458
Epoch 13/16


Metrics after epoch 13:
val.encoder_mse: 0.0097
val.decoder_loss: 0.3323
val.decoder_acc: 0.8562
val.cover_score: 0.0233
val.generated_score: 0.0477
val.ssim: 0.7247
val.psnr: 26.8260
val.bpp: 2.1374
train.encoder_mse: 0.0178
train.decoder_loss: 0.3372
train.decoder_acc: 0.8553
train.cover_score: 0.0240
train.generated_score: 0.0479
Epoch 14/16


Metrics after epoch 14:
val.encoder_mse: 0.0098
val.decoder_loss: 0.3180
val.decoder_acc: 0.8624
val.cover_score: 0.0224
val.generated_score: 0.0459
val.ssim: 0.7319
val.psnr: 26.4237
val.bpp: 2.1746
train.encoder_mse: 0.0141
train.decoder_loss: 0.3193
train.decoder_acc: 0.8637
train.cover_score: 0.0241
train.generated_score: 0.0492
Epoch 15/16


Metrics after epoch 15:
val.encoder_mse: 0.0094
val.decoder_loss: 0.3029
val.decoder_acc: 0.8694
val.cover_score: 0.0224
val.generated_score: 0.0415
val.ssim: 0.7257
val.psnr: 26.6394
val.bpp: 2.2165
train.encoder_mse: 0.0126
train.decoder_loss: 0.3091
train.decoder_acc: 0.8692
train.cover_score: 0.0239
train.generated_score: 0.0483
Epoch 16/16


Metrics after epoch 16:
val.encoder_mse: 0.0079
val.decoder_loss: 0.2975
val.decoder_acc: 0.8725
val.cover_score: 0.0240
val.generated_score: 0.0456
val.ssim: 0.7488
val.psnr: 27.4346
val.bpp: 2.2347
train.encoder_mse: 0.0155
train.decoder_loss: 0.2989
train.decoder_acc: 0.8735
train.cover_score: 0.0236
train.generated_score: 0.0444


In [6]:
# BASIC ENCODER, DECODER
# DENSE CRITIC
# Initialize data loader
batch_size = 16 # 32-image batches
train_loader = create_data_loader(cover_imgs, cover_imgs, transform=None, shuffle=True, batch_size=batch_size)
validate_loader = create_data_loader(validate_imgs, validate_imgs, transform=None, shuffle=False, batch_size=batch_size)


# Initialize agents and values
data_depth = 3 # typical for RBG values
hidden_size = 64 #


encoder = BasicMishEncoder(data_depth, hidden_size)
decoder = BasicMishDecoder(data_depth, hidden_size)
critic = DenseMishCritic(hidden_size)


# Initialize the GAN model
model = MISHSteganoGAN(
    data_depth=data_depth,
    encoder=encoder,
    decoder=decoder,
    critic=critic,
    cuda=True,
    log_dir=log_dir,
    verbose=True
)
# print(model)

model.fit(train_loader, validate_loader, epochs=16)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 148MB/s]


Using CUDA device.
Model initialized with data depth 3
Epoch 1/16


Metrics after epoch 1:
val.encoder_mse: 0.0267
val.decoder_loss: 0.6566
val.decoder_acc: 0.6572
val.cover_score: 0.0291
val.generated_score: 0.0320
val.ssim: 0.5802
val.psnr: 22.0618
val.bpp: 0.9430
train.encoder_mse: 0.0787
train.decoder_loss: 0.6818
train.decoder_acc: 0.5937
train.cover_score: 0.0415
train.generated_score: 0.0454
Epoch 2/16


Metrics after epoch 2:
val.encoder_mse: 0.0199
val.decoder_loss: 0.6253
val.decoder_acc: 0.6863
val.cover_score: 0.0199
val.generated_score: 0.0503
val.ssim: 0.6651
val.psnr: 23.4086
val.bpp: 1.1176
train.encoder_mse: 0.0330
train.decoder_loss: 0.6390
train.decoder_acc: 0.6732
train.cover_score: 0.0257
train.generated_score: 0.0446
Epoch 3/16


Metrics after epoch 3:
val.encoder_mse: 0.0170
val.decoder_loss: 0.5940
val.decoder_acc: 0.7053
val.cover_score: 0.0174
val.generated_score: 0.0605
val.ssim: 0.6871
val.psnr: 24.2778
val.bpp: 1.2315
train.encoder_mse: 0.0237
train.decoder_loss: 0.6077
train.decoder_acc: 0.6958
train.cover_score: 0.0182
train.generated_score: 0.0562
Epoch 4/16


...

Metrics after epoch 12:
val.encoder_mse: 0.0096
val.decoder_loss: 0.3579
val.decoder_acc: 0.8405
val.cover_score: 0.0678
val.generated_score: 0.0839
val.ssim: 0.7354
val.psnr: 26.8020
val.bpp: 2.0429
train.encoder_mse: 0.0149
train.decoder_loss: 0.3698
train.decoder_acc: 0.8384
train.cover_score: 0.0817
train.generated_score: 0.0852
Epoch 13/16


Metrics after epoch 13:
val.encoder_mse: 0.0093
val.decoder_loss: 0.3444
val.decoder_acc: 0.8480
val.cover_score: 0.0593
val.generated_score: 0.0643
val.ssim: 0.7415
val.psnr: 26.8172
val.bpp: 2.0883
train.encoder_mse: 0.0152
train.decoder_loss: 0.3497
train.decoder_acc: 0.8482
train.cover_score: 0.0707
train.generated_score: 0.0810
Epoch 14/16


Metrics after epoch 14:
val.encoder_mse: 0.0111
val.decoder_loss: 0.3226
val.decoder_acc: 0.8585
val.cover_score: 0.0511
val.generated_score: 0.0538
val.ssim: 0.7073
val.psnr: 25.8606
val.bpp: 2.1512
train.encoder_mse: 0.0125
train.decoder_loss: 0.3346
train.decoder_acc: 0.8563
train.cover_score: 0.0566
train.generated_score: 0.0798
Epoch 15/16


Metrics after epoch 15:
val.encoder_mse: 0.0092
val.decoder_loss: 0.3212
val.decoder_acc: 0.8601
val.cover_score: 0.0538
val.generated_score: 0.0588
val.ssim: 0.7329
val.psnr: 26.9760
val.bpp: 2.1604
train.encoder_mse: 0.0160
train.decoder_loss: 0.3228
train.decoder_acc: 0.8615
train.cover_score: 0.0614
train.generated_score: 0.0769
Epoch 16/16


Metrics after epoch 16:
val.encoder_mse: 0.0082
val.decoder_loss: 0.3129
val.decoder_acc: 0.8644
val.cover_score: 0.0504
val.generated_score: 0.0575
val.ssim: 0.7580
val.psnr: 27.4601
val.bpp: 2.1864
train.encoder_mse: 0.0134
train.decoder_loss: 0.3114
train.decoder_acc: 0.8672
train.cover_score: 0.0552
train.generated_score: 0.0674


In [7]:
# BASIC ENCODER, DECODER
# RESIDUAL CRITIC
# Initialize data loader
batch_size = 16 # 32-image batches
train_loader = create_data_loader(cover_imgs, cover_imgs, transform=None, shuffle=True, batch_size=batch_size)
validate_loader = create_data_loader(validate_imgs, validate_imgs, transform=None, shuffle=False, batch_size=batch_size)


# Initialize agents and values
data_depth = 3 # typical for RBG values
hidden_size = 64 #


encoder = BasicMishEncoder(data_depth, hidden_size)
decoder = BasicMishDecoder(data_depth, hidden_size)
critic = ResidualMishCritic(hidden_size)


# Initialize the GAN model
model = MISHSteganoGAN(
    data_depth=data_depth,
    encoder=encoder,
    decoder=decoder,
    critic=critic,
    cuda=True,
    log_dir=log_dir,
    verbose=True
)
# print(model)

model.fit(train_loader, validate_loader, epochs=16)


Using CUDA device.
Model initialized with data depth 3
Epoch 1/16


Metrics after epoch 1:
val.encoder_mse: 0.0314
val.decoder_loss: 0.6683
val.decoder_acc: 0.6296
val.cover_score: 0.0309
val.generated_score: 0.0310
val.ssim: 0.5786
val.psnr: 21.2219
val.bpp: 0.7776
train.encoder_mse: 0.0956
train.decoder_loss: 0.6872
train.decoder_acc: 0.5750
train.cover_score: 0.0707
train.generated_score: 0.1048
Epoch 2/16


Metrics after epoch 2:
val.encoder_mse: 0.0224
val.decoder_loss: 0.6395
val.decoder_acc: 0.6703
val.cover_score: 0.0438
val.generated_score: 0.0490
val.ssim: 0.6441
val.psnr: 22.7494
val.bpp: 1.0216
train.encoder_mse: 0.0341
train.decoder_loss: 0.6516
train.decoder_acc: 0.6533
train.cover_score: 0.0519
train.generated_score: 0.0658
Epoch 3/16


Metrics after epoch 3:
val.encoder_mse: 0.0183
val.decoder_loss: 0.6201
val.decoder_acc: 0.6864
val.cover_score: 0.1272
val.generated_score: 0.1275
val.ssim: 0.6975
val.psnr: 23.8356
val.bpp: 1.1182
train.encoder_mse: 0.0286
train.decoder_loss: 0.6269
train.decoder_acc: 0.6798
train.cover_score: 0.0558
train.generated_score: 0.0797
Epoch 4/16


Metrics after epoch 4:
val.encoder_mse: 0.0155
val.decoder_loss: 0.5921
val.decoder_acc: 0.7040
val.cover_score: 0.1248
val.generated_score: 0.1259
val.ssim: 0.7188
val.psnr: 24.5079
val.bpp: 1.2242
train.encoder_mse: 0.0254
train.decoder_loss: 0.6035
train.decoder_acc: 0.6963
train.cover_score: 0.1259
train.generated_score: 0.1265
Epoch 5/16


...

Metrics after epoch 12:
val.encoder_mse: 0.0102
val.decoder_loss: 0.3859
val.decoder_acc: 0.8288
val.cover_score: 0.0976
val.generated_score: 0.1094
val.ssim: 0.7457
val.psnr: 26.5303
val.bpp: 1.9725
train.encoder_mse: 0.0145
train.decoder_loss: 0.3891
train.decoder_acc: 0.8279
train.cover_score: 0.0843
train.generated_score: 0.1076
Epoch 13/16


Metrics after epoch 13:
val.encoder_mse: 0.0106
val.decoder_loss: 0.3664
val.decoder_acc: 0.8376
val.cover_score: 0.0666
val.generated_score: 0.1239
val.ssim: 0.7350
val.psnr: 26.1523
val.bpp: 2.0253
train.encoder_mse: 0.0158
train.decoder_loss: 0.3700
train.decoder_acc: 0.8378
train.cover_score: 0.0650
train.generated_score: 0.0985
Epoch 14/16


Metrics after epoch 14:
val.encoder_mse: 0.0098
val.decoder_loss: 0.3558
val.decoder_acc: 0.8442
val.cover_score: 0.0213
val.generated_score: 0.0233
val.ssim: 0.7494
val.psnr: 26.7529
val.bpp: 2.0651
train.encoder_mse: 0.0147
train.decoder_loss: 0.3550
train.decoder_acc: 0.8462
train.cover_score: 0.0221
train.generated_score: 0.0280
Epoch 15/16


Metrics after epoch 15:
val.encoder_mse: 0.0088
val.decoder_loss: 0.3300
val.decoder_acc: 0.8564
val.cover_score: 0.0268
val.generated_score: 0.0306
val.ssim: 0.7470
val.psnr: 27.3100
val.bpp: 2.1385
train.encoder_mse: 0.0142
train.decoder_loss: 0.3373
train.decoder_acc: 0.8554
train.cover_score: 0.0232
train.generated_score: 0.0260
Epoch 16/16


Metrics after epoch 16:
val.encoder_mse: 0.0086
val.decoder_loss: 0.3182
val.decoder_acc: 0.8619
val.cover_score: 0.0436
val.generated_score: 0.0644
val.ssim: 0.7425
val.psnr: 27.3927
val.bpp: 2.1711
train.encoder_mse: 0.0134
train.decoder_loss: 0.3205
train.decoder_acc: 0.8632
train.cover_score: 0.0313
train.generated_score: 0.0448
